## Load Packages

In [1]:
# # Install tempo-embeddings from GitHub
# # This can also refer to a specific version or branch

%pip install -e ..

# %pip install --upgrade pip  # Required for properly resolving dependencies
# %pip uninstall -y tempo_embeddings  # Remove existing installation
# %pip install --upgrade git+https://github.com/Semantics-of-Sustainability/tempo-embeddings.git

Obtaining file:///Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for tempo_embeddings (pyproject.toml) ... done
  Created wheel for tempo_embeddings: filename=tempo_embeddings-0.0.1-0.editable-py3-none-any.whl size=9561 sha256=52f6471b83c8d908b8a22599531703d61126112d08412fd91499540a0a8a517c
  Stored in directory: /private/var/folders/d8/j5_fyf8941j_492zvf8948y40000gn/T/pip-ephem-wheel-cache-pc0avl6y/wheels/9d/dd/1c/d0e8ab6bde11a84556c00bd645333aaad0b90e2c38e11b0ed1
Successfully built tempo_embeddings
  Attempting uninstall: tempo_embeddings
    Found existing installation: tempo_embeddings 0.0.1
    Uninstalling tempo_embeddings-0.0.1:
      Successfully uninstalled tempo_embeddings-0.0.1

[notice] A new release of pip is av

In [2]:
# make sure installation has succeeded

import logging
from importlib import reload

reload(logging)
logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)

In [3]:
%load_ext autoreload

In [4]:
try:
    import google.colab  # noqa: F401

    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False

In [5]:
from pathlib import Path

from tempo_embeddings.text.corpus import Corpus

Using corpus directory: '/Volumes/i-lab.data.uu.nl/research-semantics-of-sustainability/semantics-of-sustainability/data'


## Load Database Manager

The `db_path` parameter should point to the directory where the database is, so the original configuration and records are loaded. The database was created using the notebook `1_compute_embeddings_nl.ipynb`. If the given path does not exist, a new EMPTY database will be created there. 

A bigger `batch_size` could make the search faster but if it is too big you might run out of memory.

In [6]:
import weaviate
from tempo_embeddings.embeddings.model import SentenceTransformerModelWrapper
from tempo_embeddings.embeddings.weaviate_database import WeaviateDatabaseManager
from tempo_embeddings.settings import DEFAULT_LANGUAGE_MODEL

# Here we load only the ANP collection because metadata field names diverge across datasets
db = WeaviateDatabaseManager(
    client=weaviate.connect_to_local(port=8087),
    model=SentenceTransformerModelWrapper.from_pretrained(DEFAULT_LANGUAGE_MODEL),
)

/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
02:57:58 INFO:HTTP Request: GET http://localhost:8087/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
02:57:58 INFO:HTTP Request: GET http://localhost:8087/v1/meta "HTTP/1.1 200 OK"
02:57:58 INFO:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
02:57:59 INFO:HTTP Request: GET http://localhost:8087/v1/schema/TempoEmbeddings "HTTP/1.1 200 OK"


## Choose from the available Collections in the Database

In [7]:
from ipywidgets import widgets

existing_colls = db.get_available_collections()
collection_selector = widgets.Dropdown(
    options=existing_colls,
    description="Choose a Collection:",
    disabled=False,
    style={"description_width": "initial"},
)
collection_selector

02:57:59 INFO:HTTP Request: POST http://localhost:8087/v1/graphql "HTTP/1.1 200 OK"


Dropdown(description='Choose a Collection:', options=('NRC', 'ANP'), style=DescriptionStyle(description_width=…

## Show number of records in the selected collection

In [8]:
collection = collection_selector.value
print(
    f"\nCollection '{collection_selector.value}' has {db.get_collection_count(collection)} records"
)

02:57:59 INFO:HTTP Request: POST http://localhost:8087/v1/graphql "HTTP/1.1 200 OK"



Collection 'NRC' has 61 records


## Create Sub-Corpus

To make the processing and visualization easier, we will create a new `Corpus` comprising only a subet of the original Collection. This corpus will contain only the records of interest. This is done by querying the database with keyword and metadata constraints. In this example we allow to look for:

- **Filter Terms:** retrieve only passages that contain exactly the given keywords.
- **Year Range:** retrieve only the records which are inside the provided years
- **Neighbors:** This indicates how much to *expand* the search into more datapoints. The idea is to retrieve the *top_k* neighbors of the initially retrieved passages. Ideally this will give related passages that did not mention any of the keywords explicitly.

In [13]:
widget_year_range = widgets.IntRangeSlider(
    description="Year Range: ",
    min=1800,
    max=2020,
    step=1,
    value=(1800, 1984),
    style={"description_width": "initial"},
    layout=widgets.Layout(width="400px"),
)
widget_terms = widgets.Text(
    description="Filter Terms (comma separated)",
    value="duurzaam",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="600px"),
)
widget_neighbors = widgets.IntSlider(
    description="Expand Neighborhood Size: ",
    min=0,
    max=10,
    value=5,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="400px"),
)

### Display the Widgets to choose the parameters

In [14]:
display(widget_terms)
display(widget_year_range)
display(widget_neighbors)

Text(value='duurzaam', description='Filter Terms (comma separated)', layout=Layout(width='600px'), style=TextS…

IntRangeSlider(value=(1800, 1984), description='Year Range: ', layout=Layout(width='400px'), max=2020, min=180…

IntSlider(value=5, description='Expand Neighborhood Size: ', layout=Layout(width='400px'), max=10, style=Slide…

### Execute the Search

No need to move the code manually here. All parameters are grabbed from the widget values

In [15]:
# Unpack values form Widget
year_from, year_to = widget_year_range.value
FILTER_TERMS = [s.strip() for s in widget_terms.value.split(",")]
# Execute Database Query
where_range = {"year_from": year_from, "year_to": year_to}
print(f"Searching terms {FILTER_TERMS} between year {year_from} and {year_to}")
corpus = db.get_corpus(
    collection,
    filter_words=FILTER_TERMS,
    where_obj=where_range,
    include_embeddings=True,
    limit=10000,
)
print(f"Found {len(corpus)} items that match!")

Searching terms ['duurzaam'] between year 1800 and 1984
Found 37 items that match!


### Execute the search for neighborhoods

For each `Passage` in the `Corpus` created with the search result, we will find *k* neighbors and add them to the original `Corpus`.

In [16]:
import statistics


def get_corpus_with_neighborhoods(collection, corpus, k_neighbors):
    all_neighbors = []
    all_distances = []
    for p in corpus.passages:
        neighbors = db.query_vector_neighbors(
            collection, vector=p.embedding, k_neighbors=k_neighbors
        )
        for passage, distance in neighbors:
            all_neighbors.append(passage)
            all_distances.append(distance)
    print(f"Total Datapoints in the neighborhoods = {len(all_neighbors)}")
    print(
        f"Distance Info: Max = {max(all_distances)} | Min = {min(all_distances)} | Average = {statistics.mean(all_distances)}"
    )
    # Join original passages + new found neighbors
    all_passages = corpus.passages + all_neighbors
    corpus = Corpus(all_passages)
    corpus.embeddings = db.compress_embeddings(corpus)
    print(corpus.embeddings.shape)
    return corpus


corpus = get_corpus_with_neighborhoods(
    collection, corpus, k_neighbors=widget_neighbors.value
)

Total Datapoints in the neighborhoods = 185
Distance Info: Max = 0.6647406220436096 | Min = -8.344650268554688e-07 | Average = 0.3050347498945288
UMAP( verbose=True)
Tue Aug  6 14:58:29 2024 Construct fuzzy simplicial set
Tue Aug  6 14:58:29 2024 Finding Nearest Neighbors


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Tue Aug  6 14:58:30 2024 Finished Nearest Neighbor Search
Tue Aug  6 14:58:31 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Tue Aug  6 14:58:32 2024 Finished embedding
(222, 2)


## Cluster the Corpus

### Choose Minimum Cluster Size

In [17]:
min_cluster_size_widget = widgets.IntSlider(
    value=10,
    min=5,
    max=len(corpus) // 2,
    step=1,
    description="Minimum Cluster Size:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style={"description_width": "initial"},
)

In [18]:
min_cluster_size_widget

IntSlider(value=10, continuous_update=False, description='Minimum Cluster Size:', max=111, min=5, style=Slider…

### Execute Clustering

In [19]:
# Arguments: min_cluster_size=10, cluster_selection_epsilon=0.1, ...
# See https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html for full list

# e.g. min_samples=10, cluster_selection_epsilon=0.2, cluster_selection_method="leaf"
clusters = corpus.cluster(
    min_cluster_size=min_cluster_size_widget.value, cluster_selection_epsilon=0.1
)
print(
    f"Found {len(clusters)} clusters in the corpus. (min cluster size is {min_cluster_size_widget.value})"
)

if len(clusters) > 100:
    raise ValueError(
        "Seems like you have too many clusters! Try with a bigger value for min_cluster_size to avoid memory issues"
    )

for c in clusters:
    print(len(c.passages), c)

Found 10 clusters in the corpus. (min cluster size is 10)
27 Subcorpus(4, [0, 8, 19, 34, 37, 38, 39, 40, 41, 74])
41 Subcorpus(7, [1, 10, 16, 23, 26, 29, 31, 42, 43, 44])
31 Subcorpus(1, [2, 11, 21, 22, 47, 48, 49, 56, 61, 63])
11 Subcorpus(3, [3, 6, 18, 52, 53, 64, 67, 68, 127, 128])
32 Subcorpus(2, [4, 5, 15, 33, 45, 57, 58, 59, 60, 62])
19 Subcorpus(5, [7, 14, 24, 30, 35, 51, 72, 73, 107, 108])
17 Subcorpus(6, [9, 25, 27, 36, 82, 84, 91, 99, 126, 154])
10 Subcorpus(-1, [12, 17, 90, 97, 98, 118, 119, 122, 123, 184])
17 Subcorpus(0, [13, 32, 50, 54, 69, 80, 85, 102, 103, 129])
17 Subcorpus(8, [20, 28, 83, 86, 134, 135, 137, 138, 139, 164])


### Load Stopwords to avoid including them in the Cluster Labels

In [20]:
stopwords_file = Path("stopwords-filter-nl.txt")

with open(stopwords_file.absolute(), "rt") as f:
    stopwords = set(f.read().splitlines())

stopwords.update(
    {
        "wij",
        "we",
        "moeten",
        "heer",
        "mevrouw",
        "minister",
        "voorzitter",
        "gaat",
        "wel",
        "den",
    }
)

### Save Cluster Passages for inspection

In [21]:
import os

if not os.path.exists("clusters"):
    os.makedirs("clusters")

selected_metadata = ["year"]

all_clusters_records, df_cluster_labels, df_cluster_meta = [], [], []
for cluster in clusters:
    cluster.set_topic_label(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=5)
    df = cluster.to_dataframe()
    centroid = cluster.centroid()
    label = cluster.label
    cluster_size = len(cluster.passages)
    # Compute Cluster Stats as a Subcorpus
    top_words = " ".join(
        cluster.top_words(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=10)
    )
    all_clusters_records.append((f"{label}\t{cluster_size}\t{centroid}\t{top_words}\n"))
    df_cluster_labels.append(cluster.label)
    df_cluster_meta.append(df[selected_metadata])
    # Save the Cluster Passages in a File
    file_prefix = f"cluster_{year_from}_{year_to}_{cluster.label.replace('; ', '_')}"
    df.to_csv(f"clusters/{file_prefix}.tsv", sep="\t", index=False)
    print(cluster)

Subcorpus('betrekkingen; bladen; frankrijk; huwelijk; werkzaamheid', [0, 8, 19, 34, 37, 38, 39, 40, 41, 74])
Subcorpus('eene; goed; vele; ziekte; zullen', [1, 10, 16, 23, 26, 29, 31, 42, 43, 44])
Subcorpus('evenzeer; kalmte; rust; wanneer; werk', [2, 11, 21, 22, 47, 48, 49, 56, 61, 63])
Subcorpus('handel; middelen; minder; omslagtige; verbeterd', [3, 6, 18, 52, 53, 64, 67, 68, 127, 128])
Subcorpus('eene; grondwet; hare; herziening; noch', [4, 5, 15, 33, 45, 57, 58, 59, 60, 62])
Subcorpus('echter; geene; heden; jegens; toegevendheid', [7, 14, 24, 30, 35, 51, 72, 73, 107, 108])
Subcorpus('dezer; gouverneur; lid; staten; zoo', [9, 25, 27, 36, 82, 84, 91, 99, 126, 154])
Subcorpus('Outliers', [12, 17, 90, 97, 98, 118, 119, 122, 123, 184])
Subcorpus('evenzeer; kalmte; rust; wanneer; werk', [13, 32, 50, 54, 69, 80, 85, 102, 103, 129])
Subcorpus('20sten; mei; moge; oprigting; zwolle', [20, 28, 83, 86, 134, 135, 137, 138, 139, 164])


/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [22]:
with open(f"clusters/clusters_all_{year_from}_{year_to}.csv", "wt") as f:
    f.write("Label\tSize\tCentroid\tTopWords\n")
    for rec in all_clusters_records:
        f.write(rec)

### Plot Cluster Content Distributions

In [23]:
import logging

import matplotlib.pyplot as plt
from ipywidgets import interact

logging.getLogger("matplotlib").setLevel(logging.WARNING)


def plot_cluster_distribution(cluster_index, column_name):
    df = df_cluster_meta[cluster_index]
    cluster_name = df_cluster_labels[cluster_index]

    plt.figure(figsize=(10, 6))
    plt.hist(df[column_name], edgecolor="black")
    plt.xlabel(f"{column_name}", fontsize=14)
    plt.ylabel("Frequency", fontsize=14)
    plt.title(f"{cluster_name}", fontsize=16)
    plt.show()


cluster_selector = widgets.Dropdown(
    options=[(lbl, i) for i, lbl in enumerate(df_cluster_labels)],
    description="Choose a Cluster:",
    disabled=False,
    style={"description_width": "initial"},
)

variable_selector = widgets.Dropdown(
    options=selected_metadata,
    description="Choose a Column to Plot:",
    disabled=False,
    style={"description_width": "initial"},
)

interact(
    plot_cluster_distribution,
    cluster_index=cluster_selector,
    column_name=variable_selector,
)

/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/ipywidgets/widgets/interaction.py:43: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  from ipykernel.pylab.backend_inline import flush_figures


interactive(children=(Dropdown(description='Choose a Cluster:', options=(('betrekkingen; bladen; frankrijk; hu…

<function __main__.plot_cluster_distribution(cluster_index, column_name)>

## Visualize Embeddings (All Clusters)

In [24]:
import os

from bokeh.io import output_notebook
from bokeh.plotting import show

from tempo_embeddings.visualization.bokeh import BokehInteractiveVisualizer

output_notebook()

meta_fields = corpus.metadata_fields()
meta_fields = ["year", "date", "issue"]

visualizer = BokehInteractiveVisualizer(
    *clusters, metadata_fields=meta_fields, width=2000, height=1000
)

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*"

show(visualizer.create_document)

Loading BokehJS ...

02:58:33 INFO:Starting Bokeh server version 3.3.2 (running on Tornado 6.3.2)
02:58:33 INFO:User authentication hooks NOT provided (default user enabled)
